In [1]:
import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.stats import mode
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils.validation import check_is_fitted
from sklearn.cluster import SpectralClustering
from scipy.io import loadmat

#Please use in all section ex

In [51]:
#part of code from https://github.com/Patrick-Louden/Ising-Model/blob/master/ising.py

import math
import random


class Ising(skl.base.BaseEstimator, skl.base.TransformerMixin):
    def __init__(self,n_spins = 200, temperature = 1.0, sparsity = 100, seed = 117, p=10, overlap = 0.8):
        self.temperature = temperature
        self.n_spins = n_spins
        self.sparsity = sparsity
        self.n_C = np.ceil(np.log(n_spins)/sparsity).astype(int) #defined as in gardner, where C << logN
        self.p = p #number of patterns
        
        #choose initial condition to align with one pattern (eg. 0.8 aligned)
        self.spins = np.random.randint(2,size = (self.n_spins, 1))*2 -1 #initialize spin configuration, column vector
        self.seed = seed
        self.overlap = overlap #quantify the amount of overal you want with the first pattern and the inital spins
    
    def getSpins(self):
        return self.spins
    
    
    def randomPatterns(self):
        np.random.seed(self.seed)
        pattern_tensor = np.random.randint(2,size = (1,self.n_spins,self.p-1))*2 -1
        
        aligned_pattern = np.copy(self.spins).reshape(1,self.n_spins,1)
        idx_no_align = random.sample(range(self.n_spins),int(self.overlap*self.n_spins))
        aligned_pattern[:,idx_no_align,:] = aligned_pattern[:,idx_no_align,:]*-1

        pattern_tensor_full = np.concatenate((aligned_pattern, pattern_tensor), axis = 2)

        return pattern_tensor_full, aligned_pattern
        
    def randomPatternMatrix(self):
        patterns_1d, pattern = self.randomPatterns()
        pattern_matrix = np.tile(patterns_1d, (self.n_spins,1,1))
        return pattern_matrix

    def setTemperature(self,newTemperature):
        self.temperature = newTemperature  

    def getTemperature(self):
        return self.temperature

    def setConnectivity(self):
        np.random.seed(self.seed)
        '''
        Create the connectivity matrix C, where the probability of Cij = 1 is C/N. Note that Cij does not
        have to equal Cji. Also, there is no self-connectivity, so the diagonal of C is all 0's
        '''
        C_mat = np.random.binomial(1, self.n_C/self.n_spins, (self.n_spins,self.n_spins))
        np.fill_diagonal(C_mat, 0)#no self connectivity in the model
        return C_mat
    
    def setWeights(self):
        pattern_matrix = self.randomPatternMatrix()
        interactions = np.multiply(pattern_matrix, np.transpose(pattern_matrix,(1,0,2)))#look at the interactions within spins
        interacions_across_patterns = np.sum(interactions, axis = 2)#sum the interactions across patterns
        C_mat = self.setConnectivity()
        J_mat = np.multiply(C_mat, interacions_across_patterns)
        return J_mat
    
    def computeField(self):
        J_mat = self.setWeights()
        field_matrix = J_mat@self.spins
        return field_matrix
    
    def getProbabilitySpin(self):
        '''
        Define the probability that the spin when updated equals +1, 1-prob = spin equals -1.
        '''
       
        field = self.computeField()#get the field at time = t
        temp = self.getTemperature() #get the temperature T
        prob = (1/(1+np.exp(-2*field/temp)))
        return prob
    
    def updateSpin(self):
        '''
        Update spins based on field and the probability of the spin value
        '''
        np.random.seed(self.seed)

        spin_new = np.ones((self.n_spins,1))
        prob = self.getProbabilitySpin()
        rand_val = np.random.uniform(0,1) #if value is below prob, set spin to +1, else to -1
        
        for i in range(self.n_spins):
            prob_spin = prob[i]
            if rand_val > prob_spin:
                spin_new[i] = 1
        
        self.spins = np.copy(spin_new)
        
        
    def individualMean(self, idx):
        '''
        Calculate the interaction between an individual spin and the corresponding value in pattern 1
        
        Arguments:
            idx (int): index at which you want to compute the interaction
            pattern(int): pattern index with which you are looking at the overlap
           
        '''
        
        patterns, pattern = self.randomPatterns()
        individual_pattern = pattern[:,idx,:]
        
        spins = self.spins
        individual_spin = spins[idx]
        
        individual_mean = np.int(individual_pattern)*np.int(individual_spin)
        
        return individual_mean
    
    def averageMean(self):
        '''
        Calculate the averge mean field for all spins
        
        Arguments:
            pattern(int): pattern index with which you are looking at the overlap
        
        '''
        patterns, pattern = self.randomPatterns()#define the matrix of patterns and the one that will align
        normalization = 1/self.n_spins
        avg_mean = 0
        
        for idx in range(self.n_spins):
            indiv_mean = self.individualMean(idx)
            avg_mean += indiv_mean
            
        avg_mean_norm = normalization*avg_mean
        
        return avg_mean_norm 




In [45]:
IS = Ising()
prob= IS.getProbabilitySpin()
spins = IS.getSpins()
J_mat = IS.setWeights()
C_mat = IS.setConnectivity()

rand_patterns, rand_pattern = IS.randomPatterns()

indiv_mean = IS.individualMean(3)

In [50]:
prob.shape

(200, 1)

In [4]:
#Check that all probabilities are between 0 and 1

print("Probability wrong in :")
print(np.where(prob > 1))
print(np.where(prob <0))

Probability wrong in :
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [5]:
np.unique(J_mat)

array([-8, -6, -4, -2,  0,  2,  4,  6,  8, 10])

In [8]:
A = np.zeros((2,2))
B = np.ones((2,2))
C = np.concatenate((A,B), axis = 0)
print(C)

[[0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]]


[1, 2]


In [52]:
#run simulation for T = t

def run_sim(epochs, IS):
    
    patterns, aligned_pattern = IS.randomPatterns()
    init_mean = np.copy(IS.averageMean())
    means = [init_mean]
    
    for e in range(epochs):
        IS.updateSpin()
        mean_new = np.copy(IS.averageMean())
        means.append(mean_new)
    
    return means
        
    

In [55]:
IS = Ising()

means = run_sim(100,IS)

In [56]:
means

[array(-0.65),
 array(-0.65),
 array(-0.51),
 array(-0.6),
 array(-0.64),
 array(-0.53),
 array(-0.68),
 array(-0.68),
 array(-0.62),
 array(-0.67),
 array(-0.56),
 array(-0.64),
 array(-0.69),
 array(-0.59),
 array(-0.61),
 array(-0.52),
 array(-0.48),
 array(-0.53),
 array(-0.59),
 array(-0.68),
 array(-0.49),
 array(-0.65),
 array(-0.59),
 array(-0.63),
 array(-0.53),
 array(-0.51),
 array(-0.69),
 array(-0.59),
 array(-0.6),
 array(-0.59),
 array(-0.64),
 array(-0.67),
 array(-0.68),
 array(-0.6),
 array(-0.7),
 array(-0.71),
 array(-0.69),
 array(-0.64),
 array(-0.51),
 array(-0.55),
 array(-0.59),
 array(-0.57),
 array(-0.57),
 array(-0.54),
 array(-0.48),
 array(-0.53),
 array(-0.59),
 array(-0.65),
 array(-0.6),
 array(-0.64),
 array(-0.47),
 array(-0.68),
 array(-0.52),
 array(-0.67),
 array(-0.54),
 array(-0.63),
 array(-0.62),
 array(-0.6),
 array(-0.53),
 array(-0.63),
 array(-0.52),
 array(-0.56),
 array(-0.62),
 array(-0.62),
 array(-0.58),
 array(-0.51),
 array(-0.57),
 